In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

# === 1. Загрузка данных с обработкой ошибок ===
filename = 'load_stroke_compression.csv'

if not os.path.exists(filename):
    print(f"❌ Файл '{filename}' не найден. Загрузите его в текущую директорию.")
else:
    try:
        data = np.genfromtxt(filename, delimiter=',', skip_header=0, invalid_raise=False, filling_values=np.nan)
        data = data[~np.isnan(data).any(axis=1)]
        if len(data) == 0:
            raise ValueError("Файл пуст или содержит только некорректные данные.")

        displacement = data[:, 0]
        force = data[:, 1]

        print(f"✅ Успешно загружено {len(data)} точек данных.")

    except Exception as e:
        print(f"❌ Ошибка при загрузке данных: {e}")
        raise

# === 2. Класс для автоматического разделения кривой ===
class CompressionCurveAnalyzer:
    def __init__(self, displacement, force):
        self.displacement = np.array(displacement)
        self.force = np.array(force)
        self.idx_stage1_end = None
        self.idx_stage2_end = None
        self.stage1_mask = None
        self.stage2_mask = None
        self.stage3_mask = None

    def find_stage_boundaries(self):
        """Находит границы сегментов по изменению наклона (первой производной)."""
        n = len(self.force)

        # Вычисляем первую производную (наклон) — dF/dδ
        # Используем центральные разности для внутренних точек, крайние — односторонние
        slope = np.zeros(n)
        slope[0] = (self.force[1] - self.force[0]) / (self.displacement[1] - self.displacement[0])
        slope[-1] = (self.force[-1] - self.force[-2]) / (self.displacement[-1] - self.displacement[-2])
        for i in range(1, n-1):
            slope[i] = (self.force[i+1] - self.force[i-1]) / (self.displacement[i+1] - self.displacement[i-1])

        # Находим первый значимый наклон (начало упругого участка)
        # Ищем первую точку, где наклон превышает 1% от максимального наклона
        max_slope = np.max(slope)
        threshold_slope = 0.01 * max_slope  # можно менять (0.005, 0.02 и т.д.)

        # Этап 1: выборка зазоров — до первого значимого наклона
        stage1_end_idx = 0
        for i in range(len(slope)):
            if slope[i] > threshold_slope:
                stage1_end_idx = i - 1  # чтобы не включать точку с резким скачком
                break

        if stage1_end_idx < 0:
            stage1_end_idx = 0

        # Этап 2: упругая деформация — до точки, где наклон начинает падать (точка перегиба)
        # Ищем максимум наклона (самый крутой участок) — обычно это конец упругой зоны
        # Но часто максимум бывает в начале пластичности — поэтому ищем, где наклон начинает убывать
        peak_idx = np.argmax(slope)  # индекс максимального наклона

        # Проверяем, что пик не слишком близко к началу
        if peak_idx <= stage1_end_idx + 5:
            # Если пик почти сразу после начала — ищем следующий локальный максимум
            # Или просто берем 10% от длины кривой как границу
            stage2_end_idx = int(0.1 * n)
        else:
            # Ищем первую точку после пика, где наклон становится меньше 90% от пика
            # Это будет конец упругого участка
            stage2_end_idx = peak_idx
            for i in range(peak_idx + 1, len(slope)):
                if slope[i] < 0.9 * slope[peak_idx]:
                    stage2_end_idx = i - 1
                    break

        # Защита от выхода за границы
        stage2_end_idx = min(stage2_end_idx, n - 2)

        # Сохраняем границы
        self.idx_stage1_end = stage1_end_idx
        self.idx_stage2_end = stage2_end_idx

        # Создаем маски
        self.stage1_mask = np.zeros(n, dtype=bool)
        self.stage2_mask = np.zeros(n, dtype=bool)
        self.stage3_mask = np.zeros(n, dtype=bool)

        self.stage1_mask[:self.idx_stage1_end + 1] = True
        self.stage2_mask[self.idx_stage1_end + 1:self.idx_stage2_end + 1] = True
        self.stage3_mask[self.idx_stage2_end + 1:] = True

        return self.stage1_mask, self.stage2_mask, self.stage3_mask

    def plot_segments(self, figsize=(10, 6)):
        """Рисует непрерывный график с цветной сегментацией."""
        if self.stage1_mask is None:
            self.find_stage_boundaries()

        plt.figure(figsize=figsize)

        # Рисуем весь график тонкой линией (опционально)
        # plt.plot(self.displacement, self.force, color='black', linewidth=0.5, alpha=0.3)

        # Рисуем сегменты разными цветами — НЕПРЕРЫВНО
        plt.plot(self.displacement[self.stage1_mask], self.force[self.stage1_mask],
                 color='gray', linewidth=2, label='Этап 1: выборка зазоров')
        plt.plot(self.displacement[self.stage2_mask], self.force[self.stage2_mask],
                 color='green', linewidth=2, label='Этап 2: упругая деформация')
        plt.plot(self.displacement[self.stage3_mask], self.force[self.stage3_mask],
                 color='red', linewidth=2, label='Этап 3: пластическое течение')

        plt.xlabel('Перемещение, мм', fontsize=12)
        plt.ylabel('Сила, Н', fontsize=12)
        plt.title('Сегментация кривой сжатия (по наклону)', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend()
        plt.tight_layout()
        plt.show()

# === 3. Запуск анализа ===
analyzer = CompressionCurveAnalyzer(displacement, force)
analyzer.plot_segments()

❌ Файл 'load_stroke_compression.csv' не найден. Загрузите его в текущую директорию.


NameError: name 'displacement' is not defined